In [1]:
import requests
import numpy as np
import re
import json
import time
import pandas as pd
from collections import deque
from bs4 import BeautifulSoup

## Scrape usernames via popular artists

In [19]:
API_KEY=None
USER_AGENT=None
PASSWORD=None
with open("./secrets.txt", "r") as s:
    lines = s.readlines()
    API_KEY = lines[0].strip("\n")
    USER_AGENT = lines[1].strip("\n")
    PASSWORD = lines[2].strip("\n")

In [20]:
def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

In [21]:
def getTopArtists(n=50):
    payload = {
        "method": "chart.gettopartists",
        "limit": n
    }
    r = lastfm_get(payload)
    artists = r.json()['artists']["artist"]
    return [a['name'] for a in artists]

In [22]:
def getUserInfo(user):
    payload = {
        "method": "user.getinfo",
        "user": user
    }
    r = lastfm_get(payload)
    j = r.json()["user"]
    return [j["name"], j["country"], j["playcount"], j["registered"]["unixtime"], j["gender"]]

In [23]:
def getTopArtists_user(user, client, file, userid, artistids, n=50):
    periods = ["overall"]#, "1month", "12month"]
    output=[]
    for p in periods:
        payload = {
            "method": "user.gettopartists",
            "user": user,
            "period": p,
            "limit": n
        }
        r = lastfm_get(payload)
        j = r.json()
        if "topartists" in j and "artist" in j["topartists"]:
            artists = r.json()['topartists']["artist"]
            for a in artists:
                if a['name'] not in artistids:
                    artistids.append(a['name'])
                    f.write(f"{userid},{len(artistids)-1},{int(a['playcount'])}\n")
                else:
                    f.write(f"{userid},{list(artistids).index(a['name'])},{int(a['playcount'])}\n")

In [24]:
client = requests.Session()

In [30]:
# fuid = open('./udata/userids.csv', "r", encoding="utf-8")
users = pd.read_csv("./output/users_unique.csv", header=None)[0].to_numpy()[4000:]
# fuid.close()

userids = list(pd.read_csv("./udata/userids.csv", header=None)[1].to_numpy())
artistids = list(pd.read_csv("./udata/artistids.pcpsv", header=None, engine="python", sep="\|,\|")[1].to_numpy())

In [280]:
userids = []
artistids = []

In [31]:
ftop = open("./udata/top_artists_smaller.csv", "a", encoding="utf-8")
faid = open('./udata/artistids.pcpsv', "a", encoding="utf-8")
fuid = open('./udata/userids.csv', "a", encoding="utf-8")

In [290]:
users = pd.read_csv("./output/users_unique.csv", header=None)[0].to_numpy()s

In [ ]:
for u in users:
    if u not in userids:
        userids.append(u)
        uid = len(userids)-1
        fuid.write(f"{uid},{u}\n")
        print(f"fetching {u} w new uid {uid}")

    else:
        print(f"skipping {u} w uid {userids.index(u)}")
        continue
    payload = {
        "method": "user.gettopartists",
        "user": u,
        "period": "overall",
        "limit": 200
    }
    r = lastfm_get(payload)
    j = r.json()
    if "topartists" in j and "artist" in j["topartists"]:
        artists = r.json()['topartists']["artist"]
        for a in artists:
            if a['name'] not in artistids:
                artistids.append(a['name'])
                ftop.write(f"{uid},{len(artistids)-1},{int(a['playcount'])}\n")
                faid.write(f"{len(artistids)-1}|,|{a['name']}\n")
            else:
                ftop.write(f"{uid},{artistids.index(a['name'])},{int(a['playcount'])}\n")

In [ ]:
ftop.close()
fuid.close()
faid.close()

In [ ]:
avg = 0
with open("./output/top_artists.psv", "a", encoding="utf-8") as f:
    for i in range(10):
        tic=time.perf_counter()
        getTopArtists_user("aidan_bbq", client, f, n=500)
        avg+=time.perf_counter()-tic
        print(avg/10)

In [136]:
topartists = pd.read_csv("./output/top_artists.psv", sep = "\|,\|", engine="python")

In [137]:
topartists

,haimtruly,Taylor Swift,4184,overall
0,haimtruly,HAIM,2855,overall
1,haimtruly,Florence + the Machine,2542,overall
2,haimtruly,Kali Uchis,2418,overall
3,haimtruly,Lady Gaga,1606,overall
4,haimtruly,Ariana Grande,1298,overall
...,...,...,...,...
735,Erikvini3,MOMOLAND,3,1month
736,Erikvini3,NCT U,3,1month
737,Erikvini3,Pedro Sampaio,3,1month
738,Erikvini3,Rosé,3,1month


In [6]:
def getneighbors(username):
    url = f"https://www.last.fm/user/{username}/neighbours"
    resp = requests.get(url)    
    if resp.status_code==200:
        soup = BeautifulSoup(resp.text, 'html.parser')
        us = soup.find("section", {"class": "neighbours-items-section"})
        if us is not None:
            us = us.findAll("a", {"class": "user-list-link link-block-target"})
            return np.asarray([u.text for u in us])
    return np.array([])

In [7]:
payload = payload = {
    "username_or_email": USER_AGENT,
    "password": PASSWORD
}
loginurl = "https://www.last.fm/login"
client = requests.Session()
client.get(loginurl)
if 'csrftoken' in client.cookies:
    # Django 1.6 and up
    csrftoken = client.cookies['csrftoken']
else:
    # older versions
    csrftoken = client.cookies['csrf']
payload["csrfmiddlewaretoken"] = csrftoken
client.headers.update({'referer': loginurl})
client.post(loginurl, data=payload)

<Response [200]>

In [8]:
def getlisteners(artist, client, pages=[1,2,3]):
    users = []
    for p in pages:
        url = f"https://www.last.fm/music/{artist}/+listeners?page={p}"
        resp = client.get(url)
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'html.parser')
            us = soup.findAll("h3", {"class": "top-listeners-item-name"})
            users += [u.find("a").text for u in us]
    return users
us = getlisteners("$uicideboy$", client)

In [9]:
artists  = getTopArtists(n=1000)
my_artists = np.array(getTopArtists_user("aidan_bbq"))
explore = my_artists[~np.isin(my_artists,artists)]
with open("./output/toplisteners_me.csv", "a", encoding="utf-8") as f:
    for a in explore:
        print(f"on artist: {a}")
        [f.write(f"{a},{u}\n") for u in getlisteners(a,client)]

on artist: Adrianne Lenker
on artist: The Microphones
on artist: Hiatus Kaiyote
on artist: Jacob Collier
on artist: Esperanza Spalding
on artist: Elena Rozanova
on artist: Richard Wagner
on artist: Alfa Mist
on artist: Ennio Morricone
on artist: Buck Meek & Adrianne Lenker
on artist: Alon Mor
on artist: Moow
on artist: Stars of the Lid
on artist: Ethan Gruska
on artist: Noname
on artist: Tortoise
on artist: The Moody Blues
on artist: Nujabes
on artist: Peter, Paul & Mary
on artist: Grateful Dead
on artist: Kamasi Washington
on artist: Gojira
on artist: Bombino
on artist: Paul Horn
on artist: Vangelis
on artist: John Coltrane
on artist: Connan Mockasin
on artist: Lullatone
on artist: Barbara
on artist: Late Night Tales
on artist: Mdou Moctar
on artist: Simon Bolivar Symphony Orchestra of Venezuela & Gustavo Dudamel
on artist: Arthur Rubinstein
on artist: Andy Shauf
on artist: Antonio Vivaldi
on artist: Vulfpeck
on artist: black midi
on artist: Julee Cruise
on artist: Les Filles De Illig

## Generate user list

In [44]:
all_users=set([])

In [18]:
import pandas as pd

In [64]:
datafile = "./output/toplisteners.csv"
with open(datafile, 'r', encoding="utf-8") as f:
    lines = [[node for node in re.split('[,](?!\s)',edge.strip('\n'))[:2]] for edge in f.readlines() if edge[0][0] != '#']
    topls = pd.DataFrame(lines, columns = ["artist", "user"])
all_users = topls.user

In [65]:
len(all_users)

89820

In [66]:
with open("./output/users.csv", "a", encoding="utf-8") as f:
    [f.write(f"{u}, top, all\n") for u in all_users]

In [69]:
users = pd.read_csv("./output/users.csv", header=None)

In [72]:
len(users[0].unique())

156292

In [74]:
with open("./output/users_unique.csv", "w", encoding="utf-8") as f:
    [f.write(f"{u}\n") for u in users[0].unique()]

## Generate user demographics

In [ ]:
with open("./output/users_unique.csv", "r", encoding="utf-8") as f:
    with open("./output/users_info.csv", "w", encoding="utf-8") as fu:
        for user in f.readlines():
            u = getUserInfo(user)
            fu.write(f"{u[0]},{u[1]},{u[2]},{u[3]},{u[4]}\n")

## Scrape last.fm via user neighbors

In [ ]:
datafile = "./output/toplisteners_me.csv"
with open(datafile, 'r', encoding="utf-8") as f:
    lines = [[node for node in re.split('[,](?!\s)',edge.strip('\n'))[:2]] for edge in f.readlines() if edge[0][0] != '#']
    topls = pd.DataFrame(lines, columns = ["artist", "user"])

In [10]:
userheads = topls.user
searchstack = deque(userheads)
userset = set(userheads)
explored = np.array([]) 
useredges = None # to make graph of users

In [11]:
edges = None
datafile = "./output/both-neighs.csv"
with open(datafile, 'r') as f:
    lines = [[node for node in re.split(',',edge.strip('\n'))[:2]] for edge in f.readlines() if edge[0][0] != '#']
    edges = np.array(lines)

In [12]:
tot_userset = set(edges.flatten())
tot_explored = np.array(set(edges[:,0]))
explored = tot_explored

In [13]:
newf = open("./output/new-neighs.csv", "a", encoding="utf-8")
totf = open("./output/both-neighs.csv", "a", encoding="utf-8")

In [14]:
numnew10 = deque([11]) # number of new users added in the past 10 iterations
while sum(numnew10) > 10  and len(searchstack) > 0:
    curruser = searchstack.popleft()
    if np.isin(curruser, explored):
        continue
    explored = np.append(explored, curruser)

    neighs = getneighbors(curruser)
    prevlen = len(userset)
    userset = userset.union(neighs)
    
    # stoppage conditions
    numnew10.append(len(userset) - prevlen)
    if len(numnew10) > 10:
        numnew10.popleft()
        
    # keeping track of overall additions
    tot_prevlen = len(tot_userset)
    tot_userset = tot_userset.union(userset)
    print(f"explored {curruser}, found {numnew10[-1]} new users, {len(tot_userset) - tot_prevlen} totally new, {len(tot_userset)} total")
    

    edges = np.asarray([np.repeat(curruser, len(neighs)), neighs]).T
    [newf.write(f"{e[0]},{e[1]}\n") for e in edges]
    [totf.write(f"{e[0]},{e[1]}\n") for e in edges]
    
    searchstack.extend(neighs[~np.isin(neighs, explored)]) # get rid of already visited neighs, add rest to searchstack

explored sagalx, found 50 new users, 1 totally new, 59345 total
explored classicalbird, found 50 new users, 0 totally new, 59345 total
explored Arthur_Zeitter, found 50 new users, 0 totally new, 59345 total
explored Ykswo, found 50 new users, 0 totally new, 59345 total
explored sandundermyfeet, found 50 new users, 0 totally new, 59345 total
explored YaBoiRockstar, found 50 new users, 0 totally new, 59345 total
explored whitech0c0late, found 50 new users, 0 totally new, 59345 total
explored Alessandro19ds, found 0 new users, 0 totally new, 59345 total


C:\Users\barbi\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


explored staywithziam, found 50 new users, 3 totally new, 59348 total
explored Juandamejiam, found 32 new users, 0 totally new, 59348 total
explored LouisThorne23, found 47 new users, 0 totally new, 59348 total
explored Hebastian15, found 25 new users, 0 totally new, 59348 total
explored IfUSeekLucas, found 48 new users, 0 totally new, 59348 total
explored olegoleg24, found 47 new users, 0 totally new, 59348 total
explored Interspeed177, found 37 new users, 0 totally new, 59348 total
explored sajan__, found 20 new users, 0 totally new, 59348 total
explored Sumi_Licious, found 13 new users, 0 totally new, 59348 total
explored leonardopeh189, found 5 new users, 0 totally new, 59348 total
explored savosuh10, found 4 new users, 0 totally new, 59348 total
explored Willycyrus23, found 35 new users, 0 totally new, 59348 total
explored lucgsas, found 28 new users, 0 totally new, 59348 total
explored mariadr3w, found 42 new users, 1 totally new, 59349 total
explored caleblg, found 6 new users, 

KeyboardInterrupt: 

In [15]:
totf.close()
newf.close()